In [0]:
import pandas as pd
import numpy as np

# Data Exploration

In [0]:
columns = ['age','workclass','id','education','education_num','marital_status',
           'occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','Class']
# Load the Training set (adult_data.data)
train = pd.read_csv('adult_data.data', header=None, names=columns)

In [0]:
# Load the Test set (adult_test.txt)
test = pd.read_csv('adult_test.txt', header=None, names=columns, skiprows=1)

# Data Preprocessing

##Replacing '?' with 'unknown'

In [0]:
train.occupation = train.occupation.str.replace('?','unknown_occupation', regex=False)
train.workclass = train.workclass.str.replace('?','unknown_workclass', regex=False)
train.native_country = train.native_country.str.replace('?','unknown_country', regex=False)

test.occupation = test.occupation.str.replace('?','unknown_occupation', regex=False)
test.workclass = test.workclass.str.replace('?','unknown_workclass', regex=False)
test.native_country = test.native_country.str.replace('?','unknown_country', regex=False)


In [0]:
train.Class = train.Class.str.replace('<=50K', '0',  regex=False)
train.Class = train.Class.str.replace('>50K', '1', regex=False)
train.Class = train.Class.astype(int)

## Handling Missing Values

### Method 1 - Logistic Regression

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
#HammingDistance
def missingValuePrediction(col1, cat_cols, wc_unknown, wc_other, wc_other_Ytrain):
  wc_other_Xtrain = wc_other[cat_cols]
  wc_other_Xtrain = pd.get_dummies(wc_other_Xtrain)
  wc_other_Xtrain = wc_other_Xtrain.join(wc_other[['age','education_num','capital_gain','capital_loss'
                                                  ,'hours_per_week']])

  #Test data
  wc_unknown_Xtest = wc_unknown[cat_cols]
  wc_unknown_Xtest = pd.get_dummies(wc_unknown_Xtest)
  wc_unknown_Xtest = wc_unknown_Xtest.join(wc_unknown[['age','education_num','capital_gain','capital_loss'
                                                  ,'hours_per_week']])

  if(wc_unknown_Xtest.columns.contains('workclass_ Never-worked')):
    wc_unknown_Xtest = wc_unknown_Xtest.drop(columns=['workclass_ Never-worked', 'workclass_ unknown_workclass'])  
  if(wc_unknown_Xtest.columns.contains('native_country_ unknown_country')):
    wc_unknown_Xtest = wc_unknown_Xtest.drop(columns=['native_country_ unknown_country'])
    
  wc_other_Xtrain = wc_other_Xtrain[wc_unknown_Xtest.columns]

  clf = LogisticRegression().fit(wc_other_Xtrain, wc_other_Ytrain)

  wc_pred = clf.predict(wc_unknown_Xtest)
  
  wc_unknown['pred'+col1] = wc_pred.astype(int).tolist()

  return wc_unknown


#### Workclass

In [0]:
# Test
wc_unknown = train[train['workclass'].str.contains('unknown_workclass')]
print('Unknown', wc_unknown.shape)
# Train
wc_other = train[~train['workclass'].str.contains('unknown_workclass')]
print('Other', wc_other.shape)

wc_other_Ytrain = wc_other['workclass']

wc_other_Ytrain = wc_other_Ytrain.str.replace('Private', '1',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Self-emp-not-inc', '2',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Local-gov', '3',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('State-gov', '4',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Self-emp-inc', '5',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Federal-gov', '6',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Without-pay', '7',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Never-worked', '8',  regex=False)

wc_other_Ytrain = wc_other_Ytrain.astype(int)

cols = ['marital_status','occupation','relationship','race','sex','native_country']
df = missingValuePrediction('workclass', cols,
                            wc_unknown, wc_other, wc_other_Ytrain)

Unknown (1836, 15)
Other (30725, 15)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: FutureWarning: The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [0]:
df.predworkclass.value_counts()

1    1835
5       1
Name: predworkclass, dtype: int64

#### Occupation

In [0]:
# Test
wc_unknown = train[train['occupation'].str.contains('unknown_occupation')]
print('Unknown', wc_unknown.shape)
# Train
wc_other = train[~train['occupation'].str.contains('unknown_occupation')]
print('Other', wc_other.shape)

wc_other_Ytrain = wc_other['occupation']

wc_other_Ytrain = wc_other_Ytrain.str.replace('Prof-specialty', '1',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Craft-repair', '2',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Exec-managerial', '3',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Adm-clerical', '4',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Sales', '5',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Other-service', '6',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Machine-op-inspct', '7',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Transport-moving', '8',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Handlers-cleaners', '9',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Farming-fishing', '10',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Tech-support', '11',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Protective-serv', '12',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Priv-house-serv', '13',  regex=False)
wc_other_Ytrain = wc_other_Ytrain.str.replace('Armed-Forces', '14',  regex=False)

wc_other_Ytrain = wc_other_Ytrain.astype(int)

cols = ['marital_status','workclass','relationship','race','sex','native_country']
df = missingValuePrediction('occupation', cols,
                            wc_unknown, wc_other, wc_other_Ytrain)

In [0]:
df.predoccupation.value_counts()

2    1054
3     748
5      34
1       7
Name: predoccupation, dtype: int64

#### Native Country

In [0]:
country_list ={" United-States" :1," Mexico":2, " Philippines":3, " Germany":4, " Canada":5, " Puerto-Rico":6,  
       " El-Salvador":7, " India":8, " Cuba":9," England":10," Jamaica":11," South":12, " China":13, 
       " Italy":14," Dominican-Republic":15, " Vietnam":16," Guatemala":17," Japan":18," Poland":19,
       " Columbia":20," Taiwan":21, " Haiti":22, " Iran":23," Portugal":24," Nicaragua":25,
       " Peru":26, " Greece":27," France":28, " Ecuador":29, " Ireland":30, " Hong":31, 
       " Cambodia":32," Trinadad&Tobago":33, " Laos":34, " Thailand":35," Yugoslavia":36, 
       " Outlying-US(Guam-USVI-etc)":37, " Hungary":38, " Honduras":39," Scotland":40,  
       " Holand-Netherlands":41}

In [0]:
# Test
wc_unknown = train[train['native_country'].str.contains('unknown_country')]
print('Unknown', wc_unknown.shape)
# Train
wc_other = train[~train['native_country'].str.contains('unknown_country')]
print('Other', wc_other.shape)

wc_other_Ytrain = wc_other['native_country']

wc_other_Ytrain.replace(country_list, inplace=True)

wc_other_Ytrain = wc_other_Ytrain.astype(int)

cols = ['marital_status','workclass','relationship','race','sex','occupation']
df = missingValuePrediction('native_country', cols,
                            wc_unknown, wc_other, wc_other_Ytrain)

Unknown (583, 15)
Other (31978, 15)


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: FutureWarning: The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: FutureWarning: The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_ite

In [0]:
df.prednative_country.value_counts()

1    583
Name: prednative_country, dtype: int64